In [ ]:
#エッジ用のnumpy保存プログラム
import numpy as np
from PIL import Image
import cv2
import os,glob,random

outfile="./num16karnel3light128px.npz"#変更必要
max_photo=40000
photo_size=128#変更必要
x=[]
y=[]
laplace5=[[-1,-3,-4,-3,-1],
         [-3,0,6,0,-3],
         [-4,6,20,6,-4],
         [-3,0,6,0,-3],
         [-1,-3,-4,-3,-1]]
laplace3=[[1,1,1],
         [1,-8,1],
         [1,1,1]]
def main():
    glob_files("./grayhands/mine/omote",0)
    glob_files("./grayhands/mine/ura",1)
    glob_files("./grayhands/okutani/omote",2)
    glob_files("./grayhands/okutani/ura",3)
    glob_files("./grayhands/tochikubo/omote",4)
    glob_files("./grayhands/tochikubo/ura",5)
    glob_files("./grayhands/nakamura/omote",6)
    glob_files("./grayhands/nakamura/ura",7)
    glob_files("./grayhands/gotou/omote",8)
    glob_files("./grayhands/gotou/ura",9)
    glob_files("./grayhands/mimura/omote",10)
    glob_files("./grayhands/mimura/ura",11)
    glob_files("./grayhands/arai/omote",12)
    glob_files("./grayhands/arai/ura",13)
    glob_files("./grayhands/isii/omote",14)
    glob_files("./grayhands/isii/ura",15)
    #glob_files("./hands/saitouhide/omote",16)
    #glob_files("./hands/saitouhide/ura",17)
    #glob_files("./hands/saitouryou/omote",18)
    #glob_files("./hands/saitouryou/ura",19)
    #glob_files("./hands/sakuma/omote",20)
    #glob_files("./hands/sakuma/ura",21)
    #glob_files("./hands/tanaka/omote",22)
    #glob_files("./hands/tanaka/ura",23)
    #glob_files("./hands/tokuyama/omote",24)
    #glob_files("./hands/tokuyama/ura",25)
    #glob_files("./hands/watanabe/omote",26)
    #glob_files("./hands/watanabe/ura",27)
    #glob_files("./hands/yamaguchi/omote",28)
    #glob_files("./hands/yamaguchi/ura",29)

    np.savez(outfile,x=x,y=y)
    print("保存しました:"+outfile,len(x))
    print(x[0],y[0])
    
def glob_files(path,label):
    files=glob.glob(path+"/*.jpg")
    random.shuffle(files)
    num=0
    for f in files:
        if num>=max_photo: break
        num+=1
        
        #---------edgesの時のプログラム--------------
        img=cv2.imread(f)
        img_gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        karnel=np.array(laplace3,np.float64)#変更必要
        edges=cv2.filter2D(img_gray,-1,karnel)
        edges=cv2.resize(edges,(photo_size,photo_size))
        
        
        edges=np.asarray(edges)#1つの配列に格納（この場合2つ）
        x.append(edges)
        y.append(label)
if __name__=='__main__':
    main()

In [ ]:
#エッジファイルの中身を確認するプログラム
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
#写真データ読み込み
photos=np.load('./num16edges5light256.npz')#変更必要
x=photos['x']
y=photos['y']
#開始インデックス
idx=1900
#pyplotで出力
plt.figure(figsize=(10,10))
plt.axis("off")
print(x[0])
print(x[1])
'''
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.axis("off")
    #グレイスケール画像を読み込むときは、plt.grayが必要
    #plt.gray
    plt.title(y[i+idx])
    plt.imshow(x[i+idx],cmap='gray')
    #plt.savefig('./extract.jpg')
'''
#--------------------画像1枚をピックアップして保存する-----------------------------
plt.imshow(x[0])
#plt.imshow(x[1],cmap='gray')
plt.savefig('./edges128.jpg')
plt.show()

In [ ]:
%%time
#エッジ検出を用いた機械学習
import keras 
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten
from keras.layers import Conv2D,MaxPooling2D
from keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np

def def_model(in_shape, nb_classes):
    model = Sequential()
    model.add(Conv2D(32,
              kernel_size=(3, 3),
              activation='relu',
              input_shape=in_shape))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes, activation='softmax'))
    return model

def get_model(in_shape, nb_classes):
    model = def_model(in_shape, nb_classes)
    model.compile(
        loss='categorical_crossentropy',
        optimizer=RMSprop(),
        metrics=['accuracy'])
    return model


im_rows = 128 # 変更必要
im_cols = 128 # 変更必要
im_color = 1
in_shape = (im_rows, im_cols, im_color)
nb_classes = 16 #変更必要

photos = np.load('./num16edges5light256.npz')#変更必要
x = photos['x']
y = photos['y']

x = x.reshape(-1, im_rows, im_cols, im_color)
x = x.astype('float32') / 255
y = keras.utils.to_categorical(y.astype('int32'), nb_classes)

x_train, x_test, y_train, y_test = train_test_split(
    x, y, train_size=0.8)

model = get_model(in_shape, nb_classes)


hist = model.fit(x_train, y_train,
          batch_size=32,
          epochs=20,
          verbose=1,
          validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=1)
print('正解率=', score[1], 'loss=', score[0])


plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Accuracy')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Loss')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

model.save_weights('./hdf/num16karnel3light128pix.hdf5') #変更必要

In [1]:
#エッジ検出を用いた機械学習
import keras 
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten
from keras.layers import Conv2D,MaxPooling2D
from keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import cv2
from PIL import Image


def def_model(in_shape, nb_classes):
    model = Sequential()
    model.add(Conv2D(32,
              kernel_size=(3, 3),
              activation='relu',
              input_shape=in_shape))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes, activation='softmax'))
    return model


def get_model(in_shape, nb_classes):
    model = def_model(in_shape, nb_classes)
    model.compile(
        loss='categorical_crossentropy',
        optimizer=RMSprop(),
        metrics=['accuracy'])
    return model


#他人受入率用のテスト配列
omote=["saitouhide/omote","saitouryou/omote",
      "sakuma/omote","tanaka/omote","tokuyama/omote","watanabe/omote",
      "yamaguchi/omote"]
ura=["saitouhide/ura","saitouryou/ura",
      "sakuma/ura","tanaka/ura","tokuyama/ura","watanabe/ura",
      "yamaguchi/ura"]

target_omote=[]
target_ura=[]
laplace5=[[-1,-3,-4,-3,-1],
         [-3,0,6,0,-3],
         [-4,6,20,6,-4],
         [-3,0,6,0,-3],
         [-1,-3,-4,-3,-1]]
laplace3=[[1,1,1],
         [1,-8,1],
         [1,1,1]]
passpercent=[]
notpasspercent=[]
omotepercent=[]
urapercent=[]
for k in range(7):#変更必要
    for i in range(10):
        target_omote.append("./graytest/"+str(omote[k])+"/"+str(i)+".jpg")#変更必要
        target_ura.append("./graytest/"+str(ura[k])+"/"+str(i)+".jpg")#変更必要


im_rows = 128 #変更必要
im_cols = 128 #変更必要
im_color = 1 #変更必要
in_shape = (im_rows, im_cols, im_color)
nb_classes = 16 #変更必要

#他人受入率用のラベル

LABELS=["江波戸の表","江波戸の裏","奥谷君の表","奥谷君の裏",
       "栃窪先生の表","栃窪先生の裏","中村君の表","中村君の裏",
       "後藤さんの表","後藤さんの裏","三村君の表","三村君の裏",
       "新井さんの表","新井さんの裏","石井さんの表","石井さんの裏"]

model = get_model(in_shape, nb_classes)
model.load_weights('./hdf/num16karnel3light128pix.hdf5')#変更必要

def check_photo(path):
    img=cv2.imread(path)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    karnel=np.array(laplace3,np.float64)
    #img=Image.open(path)
    img=cv2.filter2D(img,-1,karnel)
    img=cv2.resize(img,(im_rows,im_cols))
    plt.imshow(img,cmap="gray")
    plt.show()

    x=np.asarray(img)
    x=x.reshape(-1,im_rows,im_cols,im_color)
    x=x/255
    
    #予測
    pre=model.predict([x])[0]#画像の予測
    idx=pre.argmax()
    #per=(pre[idx]*100)
    per=int(pre[idx]*100)
    if per<99 and ("omote" in path):
        omotepercent.append(per)
    elif per<99 and ("ura" in path):
        urapercent.append(per)
    else:
        pass
        
        
    return(idx,per)

def check_photo_str(path):
    idx,per=check_photo(path)
    if per>=99:
        print("この写真は、",LABELS[idx],"です。")
        print("可能性は、",per,"%です。")
        passpercent.append(per)
    else:
        print('\033[31m'+"可能性が",per,"%なので認証できません。"+'\033[31m')
        print("認証の結果は",LABELS[idx],"でした。")
        notpasspercent.append(per)
    
if __name__=='__main__':
    number=[9,19,29,39,49,59,69,79,89,99,109,119,129,139,149]
    for i in range(70):
        print("\nここからは"+str(i)+"番目の写真です")
        check_photo_str(target_omote[i])
        for l in range(15):
            if i==number[l]:
                print("================================================")
                print("テスト対象の人が変わります")
                print("================================================")
    for i in range(70):
        print("\nここからは"+str(i)+"番目の写真です")
        check_photo_str(target_ura[i])
        for l in range(15):
            if i==number[l]:
                print("================================================")
                print("テスト対象の人が変わります")
                print("================================================")
    print(passpercent)
    print(notpasspercent)
    print("閾値以上で識別したテスト数"+str(len(passpercent)))
    print("閾値未満で識別したテスト数"+str(len(notpasspercent)))
    print("表の手だけで閾値未満で識別したテスト数"+str(len(omotepercent)))
    print("裏の手だけで閾値未満で識別したテスト数"+str(len(urapercent)))
    totalpass=len(passpercent)/140
    totalnotpass=len(notpasspercent)/140
    omotenotpass=len(omotepercent)/70
    omotepass=(70-len(omotepercent))/70
    uranotpass=len(urapercent)/70
    urapass=(70-len(urapercent))/70
    print("================================================")
    print("未学習の手を閾値以上で識別した割合"+str(round(totalpass*100))+"%")
    print("未学習の手を閾値未満で識別した割合"+str(round(totalnotpass*100))+"%")
    print("表の手だけで未学習の手を閾値以上で識別した割合"+str(round(omotepass*100))+"%")
    print("表の手だけで未学習の手を閾値未満で識別した割合"+str(round(omotenotpass*100))+"%")
    print("裏の手だけで未学習の手を閾値以上で識別した割合"+str(round(urapass*100))+"%")
    print("裏の手だけで未学習の手を閾値未満で識別した割合"+str(round(uranotpass*100))+"%")

KeyboardInterrupt: 

In [ ]:
#エッジ検出を用いた機械学習
import keras 
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten
from keras.layers import Conv2D,MaxPooling2D
from keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import cv2
from PIL import Image


def def_model(in_shape, nb_classes):
    model = Sequential()
    model.add(Conv2D(32,
              kernel_size=(3, 3),
              activation='relu',
              input_shape=in_shape))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes, activation='softmax'))
    return model


def get_model(in_shape, nb_classes):
    model = def_model(in_shape, nb_classes)
    model.compile(
        loss='categorical_crossentropy',
        optimizer=RMSprop(),
        metrics=['accuracy'])
    return model


#本人受入率用のテスト配列

omote=["mine/omote","okutani/omote","tochikubo/omote",
       "nakamura/omote","gotou/omote","mimura/omote",
       "arai/omote","isii/omote"]
ura=["mine/ura","okutani/ura","tochikubo/ura",
       "nakamura/ura","gotou/ura","mimura/ura",
       "arai/ura","isii/ura"]

target_omote=[]
target_ura=[]
laplace5=[[-1,-3,-4,-3,-1],
         [-3,0,6,0,-3],
         [-4,6,20,6,-4],
         [-3,0,6,0,-3],
         [-1,-3,-4,-3,-1]]
laplace3=[[1,1,1],
         [1,-8,1],
         [1,1,1]]
passpercent=[]
notpasspercent=[]
omotepercent=[]
urapercent=[]
for k in range(7):#変更必要
    for i in range(10):
        target_omote.append("./graytest/"+str(omote[k])+"/"+str(i)+".jpg")#変更必要
        target_ura.append("./graytest/"+str(ura[k])+"/"+str(i)+".jpg")#変更必要


im_rows = 128 #変更必要
im_cols = 128 #変更必要
im_color = 1 #変更必要
in_shape = (im_rows, im_cols, im_color)
nb_classes = 16 #変更必要

#ラベル

LABELS=["江波戸の表","江波戸の裏","奥谷君の表","奥谷君の裏",
       "栃窪先生の表","栃窪先生の裏","中村君の表","中村君の裏",
       "後藤さんの表","後藤さんの裏","三村君の表","三村君の裏",
       "新井さんの表","新井さんの裏","石井さんの表","石井さんの裏"]

model = get_model(in_shape, nb_classes)
model.load_weights('./hdf/num16karnel3light128pix.hdf5')#変更必要

def check_photo(path):
    img=cv2.imread(path)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    karnel=np.array(laplace3,np.float64)
    #img=Image.open(path)
    img=cv2.filter2D(img,-1,karnel)
    img=cv2.resize(img,(im_rows,im_cols))
    plt.imshow(img,cmap="gray")
    plt.show()

    x=np.asarray(img)
    x=x.reshape(-1,im_rows,im_cols,im_color)
    x=x/255
    
    #予測
    pre=model.predict([x])[0]#画像の予測
    idx=pre.argmax()
    #per=(pre[idx]*100)
    per=int(pre[idx]*100)
    if per<99 and ("omote" in path):
        omotepercent.append(per)
    elif per<99 and ("ura" in path):
        urapercent.append(per)
    else:
        pass
        
        
    return(idx,per)

def check_photo_str(path):
    idx,per=check_photo(path)
    if per>=99:
        print("この写真は、",LABELS[idx],"です。")
        print("可能性は、",per,"%です。")
        passpercent.append(per)
    else:
        print('\033[31m'+"可能性が",per,"%なので認証できません。"+'\033[31m')
        print("認証の結果は",LABELS[idx],"でした。")
        notpasspercent.append(per)
    
if __name__=='__main__':
    number=[9,19,29,39,49,59,69,79,89,99,109,119,129,139,149]
    for i in range(70):
        print("\nここからは"+str(i)+"番目の写真です")
        check_photo_str(target_omote[i])
        for l in range(15):
            if i==number[l]:
                print("================================================")
                print("テスト対象の人が変わります")
                print("================================================")
    for i in range(70):
        print("\nここからは"+str(i)+"番目の写真です")
        check_photo_str(target_ura[i])
        for l in range(15):
            if i==number[l]:
                print("================================================")
                print("テスト対象の人が変わります")
                print("================================================")
    print(passpercent)
    print(notpasspercent)
    print("閾値以上で識別したテスト数"+str(len(passpercent)))
    print("閾値未満で識別したテスト数"+str(len(notpasspercent)))
    print("表の手だけで閾値未満で識別したテスト数"+str(len(omotepercent)))
    print("裏の手だけで閾値未満で識別したテスト数"+str(len(urapercent)))
    totalpass=len(passpercent)/140
    totalnotpass=len(notpasspercent)/140
    omotenotpass=len(omotepercent)/70
    omotepass=(70-len(omotepercent))/70
    uranotpass=len(urapercent)/70
    urapass=(70-len(urapercent))/70
    print("================================================")
    print("未学習の手を閾値以上で識別した割合"+str(round(totalpass*100))+"%")
    print("未学習の手を閾値未満で識別した割合"+str(round(totalnotpass*100))+"%")
    print("表の手だけで未学習の手を閾値以上で識別した割合"+str(round(omotepass*100))+"%")
    print("表の手だけで未学習の手を閾値未満で識別した割合"+str(round(omotenotpass*100))+"%")
    print("裏の手だけで未学習の手を閾値以上で識別した割合"+str(round(urapass*100))+"%")
    print("裏の手だけで未学習の手を閾値未満で識別した割合"+str(round(uranotpass*100))+"%")